In [29]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [30]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


train = pd.read_csv("train.csv")
campaign_data = pd.read_csv('campaign_data.csv')

# adds feature to indicate the cumulative count of emails a particular user has received
train['cumulative_count'] = train.groupby('user_id').cumcount()

# determines whether a user has opened and clicked at least one email
# 1 for yes, 0 for no
def sorting(a):
    avg = np.mean(a)
    if avg > 0:
        return 1
    else:
        return 0

train['has_opened'] = train.groupby('user_id')['is_open'].agg(sorting)
train['has_clicked'] = train.groupby('user_id')['is_click'].agg(sorting)

# add datetime column
train['date'] = pd.to_datetime(train['send_date'], format = "%d-%m-%Y %H:%M")
# set index to user_id so we can assign new columns easier
train.index = train.user_id
# drop extra columns
train.drop(['user_id','send_date'], axis = 1, inplace = True)
# create new column for number of emails
train['no_of_emails'] = train.groupby('user_id').count()['id']

# returns timestamp object day of week and hour of day
def weekday(a):
    return a.dayofweek

def hourofday(a):
    return a.hour

train['day_of_week']= train['date'].apply(weekday)
train['hour_of_day']= train['date'].apply(hourofday)

train.fillna(0, inplace = True)
train.reset_index(inplace=True)


def encode_feature(df, feature_name, column_to_agg, num_buckets = 30):
    
    # create name for new feture, and copy the dataframe
    bucket = feature_name + '_bucket_interval'
    bucket_val = feature_name + '_' + column_to_agg + '_bucket_avg_val'
    df_copy = df.copy()
        
    # create a new feature that contains the interval to which the observation belongs
    df_copy[bucket] = pd.qcut(df[feature_name],num_buckets, duplicates = 'drop') 
        
    # group the intervals together and calculate the mean price for each interval/bucket
    df_new = df_copy.groupby(df_copy[bucket], as_index = False).agg({column_to_agg:'mean'})
    df_new = pd.DataFrame(df_new)
    
    # specify the column names, and create a row that contains the number of the bucket for plotting
    df_new.rename(columns = {column_to_agg:bucket_val}, inplace = True)
    df_new['bucket_number'] = df_new.index + 1
    df_new.head()
    
    #join df and df_new on the new bucket_interval feature
    df_join = pd.merge(df_new, df_copy, on=bucket)
    df_join.drop([bucket, 'bucket_number'], axis = 1, inplace = True)
    return df_join

train = encode_feature(train, 'user_id', 'is_open')
train = encode_feature(train, 'user_id', 'is_click')
train = encode_feature(train, 'no_of_emails', 'is_click')
train = encode_feature(train,'no_of_emails', 'is_open')


df_merge = campaign_data.merge(train, on = 'campaign_id')
df_merge['train'] = 1

pivot_df = pd.pivot_table(df_merge, values="train", index="user_id", columns="campaign_id", aggfunc="count", fill_value=0).reset_index()
pivot_df.columns = ['user_id'] + ['campaign_' + str(col) for col in range(29,55)]

df = df_merge.merge(pivot_df, on = 'user_id')

pivot_df = pd.pivot_table(df_merge, values="train", index="user_id", columns="communication_type", aggfunc="count", fill_value=0).reset_index()
pivot_df.columns = ['user_id', 'conference_count', 'corporate_count','hackathon_count','newsletter_count','others_count','upcoming_events_count','webinar_count']
df = df.merge(pivot_df, on = 'user_id')

df['conference_percent']=df['conference_count'] / df['no_of_emails']
df['corporate_percent']=df['corporate_count'] / df['no_of_emails']
df['hackathon_percent']=df['hackathon_count'] / df['no_of_emails']
df['newsletter_percent']=df['newsletter_count'] / df['no_of_emails']
df['others_percent']=df['others_count'] / df['no_of_emails']
df['upcoming_events_percent']=df['upcoming_events_count'] / df['no_of_emails']
df['webinar_percent']=df['webinar_count'] / df['no_of_emails']

df.drop(['conference_count', 'corporate_count','hackathon_count','newsletter_count','others_count','upcoming_events_count','webinar_count'], axis = 1, inplace = True)


vectorizer = CountVectorizer(min_df = 1)

corpus = df['subject']
x = vectorizer.fit_transform(corpus)
x.toarray()
matrix1 = x.toarray()
vectorizer.vocabulary_.get('harvest')

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(matrix1)

asd = tfidf.toarray()
asd1 = pd.DataFrame(asd) 
features = vectorizer.get_feature_names() 
asd1.columns = features
train = pd.concat((df,asd1), axis = 1) 

In [31]:
train.head()

,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url,no_of_emails_is_open_bucket_avg_val,no_of_emails_is_click_bucket_avg_val,user_id_is_click_bucket_avg_val,user_id_is_open_bucket_avg_val,user_id,id,is_open,is_click,cumulative_count,has_opened,has_clicked,date,no_of_emails,day_of_week,hour_of_day,train,campaign_29,campaign_30,campaign_31,campaign_32,campaign_33,campaign_34,campaign_35,campaign_36,campaign_37,campaign_38,campaign_39,campaign_40,campaign_41,campaign_42,campaign_43,campaign_44,campaign_45,campaign_46,campaign_47,campaign_48,campaign_49,campaign_50,campaign_51,campaign_52,campaign_53,campaign_54,conference_percent,corporate_percent,hackathon_percent,newsletter_percent,others_percent,upcoming_events_percent,webinar_percent,000,2017,2018,50,action,ahead,allen,anand,and,announcing,articles,artificial,at,attend,av,before,bhu,big,biggest,bird,booz,borne,boss,business,by,can,ceo,chance,chat,chief,churn,click,codefest,coming,competitions,conference,convince,cross,data,datafest,datahack,day,days,delhi,dhs2017,dj,dr,early,emerging,ends,exciting,expert,expires,fireside,for,former,go,gramener,grow,hackathon,hackathons,hamilton,iit,in,india,innovate,inr,intelligence,is,job,join,july,just,keynote,kirk,lacs,largest,last,learning,live,look,machine,meetups,month,more,much,mumbai,ncr,new,newsletter,non,november,now,october,of,offer,on,opportunities,participate,passes,patil,peek,prediction,prizes,programmers,reasons,register,roles,save,science,scientist,sell,september,set,should,sneak,spot,stage,starts,summit,the,through,to,today,tonight,updates,upto,us,visualizing,watch,way,webinar,webinars,why,win,with,world,worth,you,your
0,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,0.106471,0.009791,0.011316,0.093078,4378,29_4378,0,0,1,1.0,0.0,2017-07-01 18:06:00,2,5,18,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.289596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.228605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.246952,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.0,0.0,0.362071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.000000
1,30,Upcoming Events,18,14,7,1,"Dear AVians,\r\n \r\nAre your eager to know wh...",[July] Data Science Expert Meetups & Competiti...,http://r.newsletters.analyticsvidhya.com/7up0e...,0.106471,0.009791,0.011316,0.093078,4378,30_4378,0,0,0,0.0,0.0,2017-07-05 14:06:00,2,2,14,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.319789,0.402681,0.0,0.0,0.0,0.251754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.402681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.402681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.272228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.271959,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.319789,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.312353
2,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,0.106471,0.009791,0.011316,0.093078,1070,29_1070,1,0,0,0.0,0.0,2017-07-01 18:14:00,1,5,18,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [32]:
train.shape

(1023191, 198)

In [33]:
test.head()

,id,campaign_id,user_id,send_date
0,63_122715,63,122715,01-02-2018 22:35
1,56_76206,56,76206,02-01-2018 08:15
2,57_96189,57,96189,05-01-2018 18:25
3,56_166917,56,166917,02-01-2018 08:15
4,56_172838,56,172838,02-01-2018 08:12


In [34]:
in_train_and_test = pd.merge(train,test, on = 'user_id', how = 'inner')

In [35]:
in_train_and_test.shape

(4408014, 201)

In [36]:
train.user_id.unique().shape

(168236,)

In [37]:
test.user_id.unique().shape

(198219,)

In [38]:
in_train_and_test.user_id.unique().shape

(145737,)

In [39]:
in_train_and_test.columns

Index(['campaign_id_x', 'communication_type', 'total_links',
       'no_of_internal_links', 'no_of_images', 'no_of_sections', 'email_body',
       'subject', 'email_url', 'no_of_emails_is_open_bucket_avg_val',
       ...
       'why', 'win', 'with', 'world', 'worth', 'you', 'your', 'id_y',
       'campaign_id_y', 'send_date'],
      dtype='object', length=201)

In [40]:
in_train_and_test.head(40)

,campaign_id_x,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url,no_of_emails_is_open_bucket_avg_val,no_of_emails_is_click_bucket_avg_val,user_id_is_click_bucket_avg_val,user_id_is_open_bucket_avg_val,user_id,id_x,is_open,is_click,cumulative_count,has_opened,has_clicked,date,no_of_emails,day_of_week,hour_of_day,train,campaign_29,campaign_30,campaign_31,campaign_32,campaign_33,campaign_34,campaign_35,campaign_36,campaign_37,campaign_38,campaign_39,campaign_40,campaign_41,campaign_42,campaign_43,campaign_44,campaign_45,campaign_46,campaign_47,campaign_48,campaign_49,campaign_50,campaign_51,campaign_52,campaign_53,campaign_54,conference_percent,corporate_percent,hackathon_percent,newsletter_percent,others_percent,upcoming_events_percent,webinar_percent,000,2017,2018,50,action,ahead,allen,anand,and,announcing,articles,artificial,at,attend,av,before,bhu,big,biggest,bird,booz,borne,boss,business,by,can,ceo,chance,chat,chief,churn,click,codefest,coming,competitions,conference,convince,cross,data,datafest,datahack,day,days,delhi,dhs2017,dj,dr,early,emerging,ends,exciting,expert,expires,fireside,for,former,go,gramener,grow,hackathon,hackathons,hamilton,iit,in,india,innovate,inr,intelligence,is,job,join,july,just,keynote,kirk,lacs,largest,last,learning,live,look,machine,meetups,month,more,much,mumbai,ncr,new,newsletter,non,november,now,october,of,offer,on,opportunities,participate,passes,patil,peek,prediction,prizes,programmers,reasons,register,roles,save,science,scientist,sell,september,set,should,sneak,spot,stage,starts,summit,the,through,to,today,tonight,updates,upto,us,visualizing,watch,way,webinar,webinars,why,win,with,world,worth,you,your,id_y,campaign_id_y,send_date
0,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,0.106471,0.009791,0.011316,0.093078,383,29_383,0,0,1,0.0,0.0,2017-07-01 18:04:00,2,5,18,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.289596,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.228605,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.36655,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.36655,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.36655,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.246952,0.0,0.0,0.000000,0.0,0.0,0.36655,0.0,0.0,0.000000,0.000000,0.362071,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.36655,0.0,0.0,0.000000,75_383,75,05-03-2018 18:13
1,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,0.106471,0.009791,0.011316,0.093078,383,29_383,0,0,1,0.0,0.0,2017-07-01 18:04:00,2,5,18,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.289596,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.228605,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.36655,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.36655,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.36655,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.246952,0.0,0.0,0.000000,0.0,0.0,0.36655

In [41]:
train_both = in_train_and_test.drop(['id_y','campaign_id_y','send_date'], axis = 1)
train_both.drop_duplicates(inplace = True)
#train_both.drop(['campaign_id_x'], axis = 1, inplace = True)
train_both.head()

,campaign_id_x,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url,no_of_emails_is_open_bucket_avg_val,no_of_emails_is_click_bucket_avg_val,user_id_is_click_bucket_avg_val,user_id_is_open_bucket_avg_val,user_id,id_x,is_open,is_click,cumulative_count,has_opened,has_clicked,date,no_of_emails,day_of_week,hour_of_day,train,campaign_29,campaign_30,campaign_31,campaign_32,campaign_33,campaign_34,campaign_35,campaign_36,campaign_37,campaign_38,campaign_39,campaign_40,campaign_41,campaign_42,campaign_43,campaign_44,campaign_45,campaign_46,campaign_47,campaign_48,campaign_49,campaign_50,campaign_51,campaign_52,campaign_53,campaign_54,conference_percent,corporate_percent,hackathon_percent,newsletter_percent,others_percent,upcoming_events_percent,webinar_percent,000,2017,2018,50,action,ahead,allen,anand,and,announcing,articles,artificial,at,attend,av,before,bhu,big,biggest,bird,booz,borne,boss,business,by,can,ceo,chance,chat,chief,churn,click,codefest,coming,competitions,conference,convince,cross,data,datafest,datahack,day,days,delhi,dhs2017,dj,dr,early,emerging,ends,exciting,expert,expires,fireside,for,former,go,gramener,grow,hackathon,hackathons,hamilton,iit,in,india,innovate,inr,intelligence,is,job,join,july,just,keynote,kirk,lacs,largest,last,learning,live,look,machine,meetups,month,more,much,mumbai,ncr,new,newsletter,non,november,now,october,of,offer,on,opportunities,participate,passes,patil,peek,prediction,prizes,programmers,reasons,register,roles,save,science,scientist,sell,september,set,should,sneak,spot,stage,starts,summit,the,through,to,today,tonight,updates,upto,us,visualizing,watch,way,webinar,webinars,why,win,with,world,worth,you,your
0,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,0.106471,0.009791,0.011316,0.093078,383,29_383,0,0,1,0.0,0.0,2017-07-01 18:04:00,2,5,18,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.289596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.228605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.36655,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.246952,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.000000,0.0,0.362071,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.36655,0.0,0.0,0.000000
2,46,Hackathon,7,3,2,1,Click Prediction\r\nData Science Hackathon\r\n...,[Starts Tonight] Click Prediction Hackathon | ...,http://r.newsletters.analyticsvidhya.com/7vj1a...,0.106471,0.009791,0.011316,0.093078,383,46_383,0,0,0,1.0,0.0,2017-09-14 13:40:00,2,3,13,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.343719,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.300052,0.0,0.0,0.0,0.0,0.0,0.0,0.29187,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.343719,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.328679,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.372247,0.0,0.000000,0.0,0.0,0.0,0.372247,0.0,0.343719,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.29187,0.0,0.00000,0.0,0.0,0.000000
4,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,0.106471,0.009791,0.011316,0.093078,8163,29_8163,0,

In [42]:
test_both = in_train_and_test.copy()
test_both.index = test_both.user_id
test_both.drop(train_both.columns, axis = 1, inplace = True)
test_both.reset_index(inplace = True)
test_both.drop_duplicates(inplace = True)
test_both.head()

,user_id,id_y,campaign_id_y,send_date
0,383,75_383,75,05-03-2018 18:13
1,383,71_383,71,19-02-2018 16:08
4,8163,57_8163,57,05-01-2018 18:27
5,8163,64_8163,64,06-02-2018 15:31
6,8163,74_8163,74,01-03-2018 20:58


In [43]:
#test_both.drop(['campaign_id_x'], inplace = True, axis = 1)
test_both.columns = ['user_id','id_y','campaign_id','send_date']
test_both.head()

,user_id,id_y,campaign_id,send_date
0,383,75_383,75,05-03-2018 18:13
1,383,71_383,71,19-02-2018 16:08
4,8163,57_8163,57,05-01-2018 18:27
5,8163,64_8163,64,06-02-2018 15:31
6,8163,74_8163,74,01-03-2018 20:58


In [44]:
test_both = pd.merge(test_both, campaign_data, on = 'campaign_id', how = 'left')
test_both.head()

,user_id,id_y,campaign_id,send_date,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url
0,383,75_383,75,05-03-2018 18:13,Upcoming Events,39,35,15,6,"[March 2018 - Hackathons, Competitions, Cash p...",[March 2018] It's Back: McKinsey Analytics Onl...,http://r.newsletters.analyticsvidhya.com/7wx2s...
1,383,71_383,71,19-02-2018 16:08,Hackathon,8,4,4,2,"Dear Hackers,\r\n \r\nGreat News for you all !...",[Hiring Hackathon] ABInBev is hiring Data Scie...,http://r.newsletters.analyticsvidhya.com/7wqhq...
2,8163,57_8163,57,05-01-2018 18:27,Upcoming Events,40,36,15,4,"[January - Hiring Hackathons, LearnUps, Contes...","[January 2018] Upcoming Hiring Hackathons, Lea...",http://r.newsletters.analyticsvidhya.com/7w43t...
3,8163,64_8163,64,06-02-2018 15:31,Upcoming Events,49,45,14,4,"[February - Hiring Hackathons, LearnUps, Conte...","[February 2018] Upcoming Hiring Hackathons, Da...",http://r.newsletters.analyticsvidhya.com/7wjn8...
4,8163,74_8163,74,01-03-2018 20:58,Newsletter,140,130,15,4,"March 2018 Newsletter\r\n \r\nDear Avian,\r\n\...","[March Newsletter] AV Android App, DataHack Pr...",http://r.newsletters.analyticsvidhya.com/7ww0u...


In [45]:
train.index = train.user_id
train.head()

,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url,no_of_emails_is_open_bucket_avg_val,no_of_emails_is_click_bucket_avg_val,user_id_is_click_bucket_avg_val,user_id_is_open_bucket_avg_val,user_id,id,is_open,is_click,cumulative_count,has_opened,has_clicked,date,no_of_emails,day_of_week,hour_of_day,train,campaign_29,campaign_30,campaign_31,campaign_32,campaign_33,campaign_34,campaign_35,campaign_36,campaign_37,campaign_38,campaign_39,campaign_40,campaign_41,campaign_42,campaign_43,campaign_44,campaign_45,campaign_46,campaign_47,campaign_48,campaign_49,campaign_50,campaign_51,campaign_52,campaign_53,campaign_54,conference_percent,corporate_percent,hackathon_percent,newsletter_percent,others_percent,upcoming_events_percent,webinar_percent,000,2017,2018,50,action,ahead,allen,anand,and,announcing,articles,artificial,at,attend,av,before,bhu,big,biggest,bird,booz,borne,boss,business,by,can,ceo,chance,chat,chief,churn,click,codefest,coming,competitions,conference,convince,cross,data,datafest,datahack,day,days,delhi,dhs2017,dj,dr,early,emerging,ends,exciting,expert,expires,fireside,for,former,go,gramener,grow,hackathon,hackathons,hamilton,iit,in,india,innovate,inr,intelligence,is,job,join,july,just,keynote,kirk,lacs,largest,last,learning,live,look,machine,meetups,month,more,much,mumbai,ncr,new,newsletter,non,november,now,october,of,offer,on,opportunities,participate,passes,patil,peek,prediction,prizes,programmers,reasons,register,roles,save,science,scientist,sell,september,set,should,sneak,spot,stage,starts,summit,the,through,to,today,tonight,updates,upto,us,visualizing,watch,way,webinar,webinars,why,win,with,world,worth,you,your
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4378,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,0.106471,0.009791,0.011316,0.093078,4378,29_4378,0,0,1,1.0,0.0,2017-07-01 18:06:00,2,5,18,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.289596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.228605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.246952,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.0,0.0,0.362071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.000000
4378,30,Upcoming Events,18,14,7,1,"Dear AVians,\r\n \r\nAre your eager to know wh...",[July] Data Science Expert Meetups & Competiti...,http://r.newsletters.analyticsvidhya.com/7up0e...,0.106471,0.009791,0.011316,0.093078,4378,30_4378,0,0,0,0.0,0.0,2017-07-05 14:06:00,2,2,14,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.319789,0.402681,0.0,0.0,0.0,0.251754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.402681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.402681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.272228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.271959,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.319789,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.312353
1070,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: 

In [46]:
only_train = train.drop(train_both.user_id.unique())
only_train.reset_index(inplace = True, drop = True)
only_train.head()

,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url,no_of_emails_is_open_bucket_avg_val,no_of_emails_is_click_bucket_avg_val,user_id_is_click_bucket_avg_val,user_id_is_open_bucket_avg_val,user_id,id,is_open,is_click,cumulative_count,has_opened,has_clicked,date,no_of_emails,day_of_week,hour_of_day,train,campaign_29,campaign_30,campaign_31,campaign_32,campaign_33,campaign_34,campaign_35,campaign_36,campaign_37,campaign_38,campaign_39,campaign_40,campaign_41,campaign_42,campaign_43,campaign_44,campaign_45,campaign_46,campaign_47,campaign_48,campaign_49,campaign_50,campaign_51,campaign_52,campaign_53,campaign_54,conference_percent,corporate_percent,hackathon_percent,newsletter_percent,others_percent,upcoming_events_percent,webinar_percent,000,2017,2018,50,action,ahead,allen,anand,and,announcing,articles,artificial,at,attend,av,before,bhu,big,biggest,bird,booz,borne,boss,business,by,can,ceo,chance,chat,chief,churn,click,codefest,coming,competitions,conference,convince,cross,data,datafest,datahack,day,days,delhi,dhs2017,dj,dr,early,emerging,ends,exciting,expert,expires,fireside,for,former,go,gramener,grow,hackathon,hackathons,hamilton,iit,in,india,innovate,inr,intelligence,is,job,join,july,just,keynote,kirk,lacs,largest,last,learning,live,look,machine,meetups,month,more,much,mumbai,ncr,new,newsletter,non,november,now,october,of,offer,on,opportunities,participate,passes,patil,peek,prediction,prizes,programmers,reasons,register,roles,save,science,scientist,sell,september,set,should,sneak,spot,stage,starts,summit,the,through,to,today,tonight,updates,upto,us,visualizing,watch,way,webinar,webinars,why,win,with,world,worth,you,your
0,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,0.106471,0.009791,0.011316,0.093078,4378,29_4378,0,0,1,1.0,0.0,2017-07-01 18:06:00,2,5,18,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.289596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.228605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.246952,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.0,0.0,0.362071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.000000
1,30,Upcoming Events,18,14,7,1,"Dear AVians,\r\n \r\nAre your eager to know wh...",[July] Data Science Expert Meetups & Competiti...,http://r.newsletters.analyticsvidhya.com/7up0e...,0.106471,0.009791,0.011316,0.093078,4378,30_4378,0,0,0,0.0,0.0,2017-07-05 14:06:00,2,2,14,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.319789,0.402681,0.0,0.0,0.0,0.251754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.402681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.402681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.272228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.271959,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.319789,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.312353
2,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,0.106471,0.009791,0.011316,0.093078,1070,29_1070,1,0,0,0.0,0.0,2017-07-01 18:14:00,1,5,18,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [47]:
only_test = test.drop(test_both.user_id.unique())
only_test.reset_index(inplace = True, drop = True)
only_test.head()

,id,campaign_id,user_id,send_date
0,63_122715,63,122715,01-02-2018 22:35
1,56_76206,56,76206,02-01-2018 08:15
2,57_96189,57,96189,05-01-2018 18:25
3,56_166917,56,166917,02-01-2018 08:15
4,56_172838,56,172838,02-01-2018 08:12


In [48]:
only_both = pd.merge(only_train, only_test, on = 'user_id', how = 'inner')
only_both.head()

,campaign_id_x,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url,no_of_emails_is_open_bucket_avg_val,no_of_emails_is_click_bucket_avg_val,user_id_is_click_bucket_avg_val,user_id_is_open_bucket_avg_val,user_id,id_x,is_open,is_click,cumulative_count,has_opened,has_clicked,date,no_of_emails,day_of_week,hour_of_day,train,campaign_29,campaign_30,campaign_31,campaign_32,campaign_33,campaign_34,campaign_35,campaign_36,campaign_37,campaign_38,campaign_39,campaign_40,campaign_41,campaign_42,campaign_43,campaign_44,campaign_45,campaign_46,campaign_47,campaign_48,campaign_49,campaign_50,campaign_51,campaign_52,campaign_53,campaign_54,conference_percent,corporate_percent,hackathon_percent,newsletter_percent,others_percent,upcoming_events_percent,webinar_percent,000,2017,2018,50,action,ahead,allen,anand,and,announcing,articles,artificial,at,attend,av,before,bhu,big,biggest,bird,booz,borne,boss,business,by,can,ceo,chance,chat,chief,churn,click,codefest,coming,competitions,conference,convince,cross,data,datafest,datahack,day,days,delhi,dhs2017,dj,dr,early,emerging,ends,exciting,expert,expires,fireside,for,former,go,gramener,grow,hackathon,hackathons,hamilton,iit,in,india,innovate,inr,intelligence,is,job,join,july,just,keynote,kirk,lacs,largest,last,learning,live,look,machine,meetups,month,more,much,mumbai,ncr,new,newsletter,non,november,now,october,of,offer,on,opportunities,participate,passes,patil,peek,prediction,prizes,programmers,reasons,register,roles,save,science,scientist,sell,september,set,should,sneak,spot,stage,starts,summit,the,through,to,today,tonight,updates,upto,us,visualizing,watch,way,webinar,webinars,why,win,with,world,worth,you,your,id_y,campaign_id_y,send_date


In [49]:
train_both.head()

,campaign_id_x,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url,no_of_emails_is_open_bucket_avg_val,no_of_emails_is_click_bucket_avg_val,user_id_is_click_bucket_avg_val,user_id_is_open_bucket_avg_val,user_id,id_x,is_open,is_click,cumulative_count,has_opened,has_clicked,date,no_of_emails,day_of_week,hour_of_day,train,campaign_29,campaign_30,campaign_31,campaign_32,campaign_33,campaign_34,campaign_35,campaign_36,campaign_37,campaign_38,campaign_39,campaign_40,campaign_41,campaign_42,campaign_43,campaign_44,campaign_45,campaign_46,campaign_47,campaign_48,campaign_49,campaign_50,campaign_51,campaign_52,campaign_53,campaign_54,conference_percent,corporate_percent,hackathon_percent,newsletter_percent,others_percent,upcoming_events_percent,webinar_percent,000,2017,2018,50,action,ahead,allen,anand,and,announcing,articles,artificial,at,attend,av,before,bhu,big,biggest,bird,booz,borne,boss,business,by,can,ceo,chance,chat,chief,churn,click,codefest,coming,competitions,conference,convince,cross,data,datafest,datahack,day,days,delhi,dhs2017,dj,dr,early,emerging,ends,exciting,expert,expires,fireside,for,former,go,gramener,grow,hackathon,hackathons,hamilton,iit,in,india,innovate,inr,intelligence,is,job,join,july,just,keynote,kirk,lacs,largest,last,learning,live,look,machine,meetups,month,more,much,mumbai,ncr,new,newsletter,non,november,now,october,of,offer,on,opportunities,participate,passes,patil,peek,prediction,prizes,programmers,reasons,register,roles,save,science,scientist,sell,september,set,should,sneak,spot,stage,starts,summit,the,through,to,today,tonight,updates,upto,us,visualizing,watch,way,webinar,webinars,why,win,with,world,worth,you,your
0,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,0.106471,0.009791,0.011316,0.093078,383,29_383,0,0,1,0.0,0.0,2017-07-01 18:04:00,2,5,18,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.289596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.228605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.36655,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.246952,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.000000,0.0,0.362071,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.36655,0.0,0.0,0.000000
2,46,Hackathon,7,3,2,1,Click Prediction\r\nData Science Hackathon\r\n...,[Starts Tonight] Click Prediction Hackathon | ...,http://r.newsletters.analyticsvidhya.com/7vj1a...,0.106471,0.009791,0.011316,0.093078,383,46_383,0,0,0,1.0,0.0,2017-09-14 13:40:00,2,3,13,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.343719,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.300052,0.0,0.0,0.0,0.0,0.0,0.0,0.29187,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.343719,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.328679,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.372247,0.0,0.000000,0.0,0.0,0.0,0.372247,0.0,0.343719,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.29187,0.0,0.00000,0.0,0.0,0.000000
4,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,0.106471,0.009791,0.011316,0.093078,8163,29_8163,0,

In [50]:
train_both['campaign_id'] = train_both['campaign_id_x']
train_both.drop(['campaign_id_x'], inplace = True, axis = 1)

In [51]:
train_both.head()

,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url,no_of_emails_is_open_bucket_avg_val,no_of_emails_is_click_bucket_avg_val,user_id_is_click_bucket_avg_val,user_id_is_open_bucket_avg_val,user_id,id_x,is_open,is_click,cumulative_count,has_opened,has_clicked,date,no_of_emails,day_of_week,hour_of_day,train,campaign_29,campaign_30,campaign_31,campaign_32,campaign_33,campaign_34,campaign_35,campaign_36,campaign_37,campaign_38,campaign_39,campaign_40,campaign_41,campaign_42,campaign_43,campaign_44,campaign_45,campaign_46,campaign_47,campaign_48,campaign_49,campaign_50,campaign_51,campaign_52,campaign_53,campaign_54,conference_percent,corporate_percent,hackathon_percent,newsletter_percent,others_percent,upcoming_events_percent,webinar_percent,000,2017,2018,50,action,ahead,allen,anand,and,announcing,articles,artificial,at,attend,av,before,bhu,big,biggest,bird,booz,borne,boss,business,by,can,ceo,chance,chat,chief,churn,click,codefest,coming,competitions,conference,convince,cross,data,datafest,datahack,day,days,delhi,dhs2017,dj,dr,early,emerging,ends,exciting,expert,expires,fireside,for,former,go,gramener,grow,hackathon,hackathons,hamilton,iit,in,india,innovate,inr,intelligence,is,job,join,july,just,keynote,kirk,lacs,largest,last,learning,live,look,machine,meetups,month,more,much,mumbai,ncr,new,newsletter,non,november,now,october,of,offer,on,opportunities,participate,passes,patil,peek,prediction,prizes,programmers,reasons,register,roles,save,science,scientist,sell,september,set,should,sneak,spot,stage,starts,summit,the,through,to,today,tonight,updates,upto,us,visualizing,watch,way,webinar,webinars,why,win,with,world,worth,you,your,campaign_id
0,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,0.106471,0.009791,0.011316,0.093078,383,29_383,0,0,1,0.0,0.0,2017-07-01 18:04:00,2,5,18,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.289596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.228605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.36655,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36655,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.246952,0.0,0.0,0.0,0.0,0.0,0.36655,0.0,0.0,0.000000,0.0,0.362071,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.36655,0.0,0.0,0.000000,29
2,Hackathon,7,3,2,1,Click Prediction\r\nData Science Hackathon\r\n...,[Starts Tonight] Click Prediction Hackathon | ...,http://r.newsletters.analyticsvidhya.com/7vj1a...,0.106471,0.009791,0.011316,0.093078,383,46_383,0,0,0,1.0,0.0,2017-09-14 13:40:00,2,3,13,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.343719,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.300052,0.0,0.0,0.0,0.0,0.0,0.0,0.29187,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.343719,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.328679,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.372247,0.0,0.000000,0.0,0.0,0.0,0.372247,0.0,0.343719,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.29187,0.0,0.00000,0.0,0.0,0.000000,46
4,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...,0.106471,0.009791,0.011316,0.093078,8163,29_8163,0,0,1,0

In [54]:
train_both.to_csv("train_both.csv")
only_test.to_csv("only_test.csv")